1. 构造 APM 计数矩阵与频率数组

In [9]:
import numpy as np

# APM 计数矩阵
n = 20
lower_triangle_values = [0,
    30, 0,
    109, 17, 0,
    154, 0, 532, 0,
    33, 10, 0, 0, 0,
    93, 120, 50, 76, 0, 0,
    266, 0, 94, 831, 0, 422, 0,
    579, 10, 156, 162, 10, 30, 112, 0,
    21, 103, 226, 43, 10, 243, 23, 10, 0,
    66, 30, 36, 13, 17, 8, 35, 0, 3, 0,
    95, 17, 37, 0, 0, 75, 15, 17, 40, 253, 0,
    57, 477, 322, 85, 0, 147, 104, 60, 23, 43, 39, 0,
    29, 17, 0, 0, 0, 20, 7, 7, 0, 57, 207, 90, 0,
    20, 7, 7, 0, 0, 0, 0, 17, 20, 90, 167, 0, 17, 0,
    345, 67, 27, 10, 10, 93, 40, 49, 50, 7, 43, 42, 4, 7, 0,
    772, 137, 432, 98, 117, 47, 86, 450, 26, 20, 32, 168, 20, 40, 269, 0,
    590, 20, 169, 57, 10, 37, 31, 50, 14, 129, 52, 200, 28, 10, 73, 696, 0,
    0, 27, 3, 0, 0, 0, 0, 0, 3, 0, 13, 0, 0, 10, 0, 17, 0, 0,
    20, 3, 36, 0, 30, 0, 10, 0, 40, 13, 23, 10, 0, 260, 0, 22, 23, 6, 0,
    365, 20, 13, 17, 33, 27, 37, 97, 30, 661, 303, 17, 77, 10, 50, 43, 186, 0, 17, 0]

APM = np.zeros((n, n))
indices = np.tril_indices(n)
APM[indices] = lower_triangle_values
APM = APM + APM.T - np.diag(np.diag(APM))

# 氨基酸出现概率
freq = np.array([0.087, 0.041, 0.040, 0.047, 0.033, 0.038, 0.050, 0.089, 0.034, 0.037,
                 0.085, 0.081, 0.015, 0.040, 0.051, 0.070, 0.058, 0.010, 0.030, 0.065])

2. 计算相对突变率 (Relative Mutability, RM)

In [ ]:
RM = np.zeros(20)
for i in range(n):
    RM[i] = np.sum(APM[i, :]) / freq[i]

RM = RM / RM[0] * 100  # 效仿论文将 Ala 的 RM 设置为 100

3. 计算未缩放突变矩阵 (M) 与缩放因子 ($\lambda$)

In [ ]:
M = np.zeros([n, n])
for i in range(n):
    for j in range(n):
        M[i, j] = APM[i, j] / np.sum(APM[:, j]) * RM[j]

total_mutation_prob = 0
for i in range(n):
    total_mutation_prob += np.sum(M[:, i] * freq[i])

lam = 0.01 / total_mutation_prob

4. 生成最终 PAM 矩阵  
将矩阵缩放至目标概率（如 1%），并填充对角线（不发生突变的概率）

In [7]:
PAM = M * lam
for i in range(n):
    # 对角线元素为 1 减去该列其他元素之和
    PAM[i, i] = 1 - np.sum(PAM[:, i])
    
print(PAM)

[[9.86672694e-01 2.32820198e-04 8.67061219e-04 1.04257215e-03 3.18187603e-04 7.78722293e-04 1.69275805e-03 2.07000699e-03 1.96527637e-04 5.67577887e-04 3.55621439e-04 2.23909795e-04 6.15162700e-04 1.59093802e-04 2.15244555e-03 3.50915471e-03 3.23673597e-03 0.00000000e+00 2.12125069e-04 1.78674577e-03]
 [1.09719863e-04 9.91370131e-01 1.35229731e-04 0.00000000e+00 9.64204859e-05 1.00480296e-03 0.00000000e+00 3.57514161e-05 9.63921269e-04 2.57989949e-04 6.36375207e-05 1.87377144e-03 3.60612617e-04 5.56828306e-05 4.18011165e-04 6.22738595e-04 1.09719863e-04 8.59106529e-04 3.18187603e-05 9.79038780e-05]
 [3.98648836e-04 1.31931445e-04 9.81974672e-01 3.60161287e-03 0.00000000e+00 4.18667899e-04 5.98192694e-04 5.57722091e-04 2.11501172e-03 3.09587938e-04 1.38505192e-04 1.26489393e-03 0.00000000e+00 5.56828306e-05 1.68452261e-04 1.96367207e-03 9.27132844e-04 9.54562810e-05 3.81825124e-04 6.36375207e-05]
 [5.63228631e-04 0.00000000e+00 4.23189513e-03 9.85932046e-01 0.00000000e+00 6.36375207e-04

Supplementary: 一步法直接计算 (化简公式)

In [8]:
PAM_direct = np.zeros([n, n])
for i in range(n):
    for j in range(n):
        # 化简公式：PAM_ij = (A_ij / sum(A)) * (0.01 / f_j)
        PAM_direct[i, j] = APM[i, j] / np.sum(APM) * 0.01 / freq[j]

for i in range(n):
    PAM_direct[i, i] = 1 - np.sum(PAM_direct[:, i])

PAM_direct = (PAM_direct * 10000).astype(int)
print(f"PAM_direct:\n{PAM_direct}")

PAM_direct:
[[9866    2    8   10    3    7   16   20    1    5    3    2    6    1   21   35   32    0    2   17]
 [   1 9913    1    0    0   10    0    0    9    2    0   18    3    0    4    6    1    8    0    0]
 [   3    1 9819   36    0    4    5    5   21    3    1   12    0    0    1   19    9    0    3    0]
 [   5    0   42 9859    0    6   52    5    4    1    0    3    0    0    0    4    3    0    0    0]
 [   1    0    0    0 9973    0    0    0    0    1    0    0    0    0    0    5    0    0    3    1]
 [   3    9    3    5    0 9875   26    1   22    0    2    5    4    0    5    2    2    0    0    1]
 [   9    0    7   56    0   35 9865    4    2    3    0    4    1    0    2    3    1    0    1    1]
 [  21    0   12   10    0    2    7 9935    0    0    0    2    1    1    3   20    2    0    0    4]
 [   0    7   17    2    0   20    1    0 9913    0    1    0    0    1    3    1    0    0    4    1]
 [   2    2    2    0    1    0    2    0    0 9872    9    1

将上述结果与原文的PAM1矩阵对照，基本一致。
![image](/assets/PAM1_reference.png)